In [10]:
import numpy as np
import pyomo.environ as pyo

## Chapter 19.3 Worksheet on Markov Decision Processes

"Introduction to Operations Research" by F. Hillier and G. Lieberman, Edition 10.

Chapter 19 covers pages 877 to 887: 
- Section 19.1 outlines the problem, 
- Section 19.2 presents a model, 
- Section 19.3 establishes a linear programming and mathematical optimization approach, 
- and Section 19.4 offers conclusions.

### Policies consist of decisions for each state i (Each policy is R, rows) 
R:= {d_0(R), d_1(R),...,d_M(R)} 
0) |



| State | Description                          |
|-------|--------------------------------------|
| 0     | Good as new                         |
| 1     | Operable - minor deterioration      |
| 2     | Operable - major deterioration      |
| 3     | Inoperable - output of unaccept


| Decision | Description                         |
|----------|-------------------------------------|
| 1        | Do nothing                         |
| 2        | Overhaul (return system to state 1) |
| 3        | Replace (return system to state 0) |ableR),...,d_M(R)} 

In [20]:
# R = np.array([[1, 1, 1, 3],
#              [1, 1, 2, 3],
#              [1, 1, 3, 3],
#              [1, 3, 3, 3]])

In [12]:
# R
# # array([[1, 1, 1, 3],
# #        [1, 1, 2, 3],
# #        [1, 1, 3, 3],
# #        [1, 3, 3, 3]])

### Cost matrix (immediate + subsequent) 

Where i ranges from 0 to M (3), and j is the target state from 0 to M (3). (C_ij)

Immediate and subsequent costs are captured from the example from 19.1-19.2 (page 883)

In [13]:
C = np.array([[0, np.NaN, np.NaN],
             [1, np.NaN, 6],
             [3, 4, 6],
             [np.NaN, np.NaN, 6]])

In [14]:
C

array([[ 0., nan, nan],
       [ 1., nan,  6.],
       [ 3.,  4.,  6.],
       [nan, nan,  6.]])

In [15]:
# Visualizing the valid transitions 

# Create a boolean mask (True where not NaN, False where NaN)
mask = ~np.isnan(C)

# Extract row indices (0-based) and depth indices (1-based for Pyomo)
row_idx, depth_idx = np.where(mask)
depth_idx += 1  # Convert to 1-based for Pyomo

# Combine indices into tuples
valid_pairs = list(zip(row_idx, depth_idx))

# Print results
print("Valid (i, k) pairs for Pyomo:", valid_pairs)
print("Boolean Mask:\n", mask)

Valid (i, k) pairs for Pyomo: [(0, 1), (1, 1), (1, 3), (2, 1), (2, 2), (2, 3), (3, 3)]
Boolean Mask:
 [[ True False False]
 [ True False  True]
 [ True  True  True]
 [False False  True]]


### Original Transition Probabilities (Base level or "Do Nothing")

Page 882 outlines Policy R_a, a "do nothing" policy. It assumes that to keep all states accessible and prevent them from absorbing into one state in the long run (steady state), the machine must be replaced when it deteriorates.

The main transition matrix starts with "do nothing" in Decision 1.

In [16]:
P0 = np.array([[0, 7/8, 1/16, 1/16],
              [0, 3/4, 1/8, 1/8],
              [0, 0, 1/2, 1/2],
              [1, 0, 0, 0]])

In [17]:
P0

array([[0.    , 0.875 , 0.0625, 0.0625],
       [0.    , 0.75  , 0.125 , 0.125 ],
       [0.    , 0.    , 0.5   , 0.5   ],
       [1.    , 0.    , 0.    , 0.    ]])

### Mapping Transition Matrix to Decision/Action

In [21]:
# Get dimensions
n_states, n_decisions = C.shape  # states, policies

# Initialize p_ijk as zeros
p_ijk = np.zeros((n_states, n_states, n_decisions))

# Set Decision 1 to use P0
p_ijk[:, :, 0] = P0  # First decision (do-nothing) gets P0

# Decision map
decision_transform = {
    2: np.array([0, 1, 0, 0]),
    3: np.array([1, 0, 0, 0])
}

# Apply transition changes only for actions > 1
for decision, replacement in decision_transform.items():
    action_idx = decision - 1  # 1-based to 0
    p_ijk[:, :, action_idx] = replacement  # Replace transition

# Print matrices for each action
for k in range(n_decisions):
    print(f"Decision {k+1}:\n{p_ijk[:, :, k]}\n")


Decision 1:
[[0.     0.875  0.0625 0.0625]
 [0.     0.75   0.125  0.125 ]
 [0.     0.     0.5    0.5   ]
 [1.     0.     0.     0.    ]]

Decision 2:
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]

Decision 3:
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]



In [ ]:
p_ijk

In [ ]:
p_ijk[:,:,0]

In [ ]:
p_ijk[:,:,1]

In [ ]:
p_ijk[:,:,2]

### Pyomo Model

In [ ]:
import pyomo.environ as pyo
import numpy as np

# Pyomo Model Function
def create_linear_program(objective_coefficients, 
                                 p_ijk,
                                 name="",
                                 debug=True):
    # Create model
    model = pyo.ConcreteModel(name=name)

    # Define sets
    n = p_ijk.shape[0]  # Number of states
    k = p_ijk.shape[2]  # Number of decisions

    model.rows = pyo.RangeSet(0, n - 1) 
    model.columns = pyo.RangeSet(0, n - 1) 
    model.depths = pyo.RangeSet(1, k)  # Depths are 1-based

    # Parameters/Variables
    # Extract valid (row, depth) pairs
    mask = ~np.isnan(objective_coefficients) # np.NaN indicate the impossible transitions
    row_idx, depth_idx = np.where(mask)
    depth_idx += 1  # Convert to 1-based for Pyomo
    valid_pairs = list(zip(row_idx, depth_idx))
    
    # Filtered Decision Variable Set
    model.valid_pairs = pyo.Set(initialize=valid_pairs)  # Only valid (i, k)

    # Decision Variables (Only for valid (i, k) pairs)
    model.y_ik = pyo.Var(model.valid_pairs, 
                         within=pyo.NonNegativeReals) # non-negativity constraints

    # Objective Coefficients (C_ik)
    def C_init(model, i, k):
        return objective_coefficients[i, k - 1]  # k is 1-based
    model.C_ik = pyo.Param(model.valid_pairs, initialize=C_init)

    # Transition Probabilities (p_ijk)
    def p_init(model, i, j, k):
        return p_ijk[i, j, k - 1]  # k is 1-based
    model.p_ijk = pyo.Param(model.rows, model.columns, model.depths,
                            initialize=p_init, default=0)

    # Objective Function
    model.objective = pyo.Objective(
        expr=sum(model.C_ik[i, k] * model.y_ik[i, k] for (i, k) in model.valid_pairs),
        sense=pyo.minimize)

    # Constraint: Σ y_ik == 1 (100% probability over valid (i, k))
    def sum_to_1(model):
        return sum(model.y_ik[i, k] for (i, k) in model.valid_pairs) == 1
    model.sum_to_1 = pyo.Constraint(rule=sum_to_1)

    # Constraint: Flow Conservation Σ_k{y_jk} - Σ_i,k{y_ik * p_ijk} == 0 ∀j
    def flow_conservation(model, j): 
        return sum(model.y_ik[j, k] for k in model.depths if (j, k) in model.valid_pairs) \
             - sum(model.y_ik[i, k] * model.p_ijk[i, j, k] 
                   for (i, k) in model.valid_pairs) == 0
    model.flow_conservation = pyo.Constraint(model.columns, rule=flow_conservation)

    return model

In [ ]:
# Create and solve model (used glpk, but gurobi or other solvers like cbc, highs, etc usable)
modelname = "Test"
model = create_linear_program(C, p_ijk, name=modelname)

solver = pyo.SolverFactory('glpk')
solver.solve(model, 
             tee=True)

In [ ]:
def display_results(model, 
                    precision=1, 
                    name = "", # model name
                    verbose = True,
                   sensitivity = False):
    print(f"Model: {model.name}")
    print(f"Objective Value: {pyo.value(model.objective):.{precision}f}")
    print("Decision Variables:")
 
    # Locate variables
    for variable in model.component_objects(pyo.Var, # used for grabbing the variables
                                            descend_into=True): # used to go sub-nests (nested russian doll)
        # Iterate
        for ii in variable:
            variable_value = pyo.value(variable[ii])
            print(f"\t{variable[ii].name} = {variable_value:.{precision}f}")
    print("\n")
    
    
    if verbose: # if verbose outputting is OK
        # Objective function:
        model.objective.pprint()
        
        print("\n")

        # All Constraints
        for constraint_name, constraint in model.component_map(pyo.Constraint).items():
            print(f"Constraint: {constraint_name}")
            constraint.pprint() # print constraint details
        print("\n------------------\n")

# Displaying results
display_results(model, 
                name = modelname,
                precision = 3, 
                verbose = True # integer answers but real valued decision variables
               )

#### Checking values

In [ ]:
round(5/7,3) # known exact values

In [ ]:
round(2/21,3) # known exact values

#### Extracting Outputs

In [ ]:
# Extract y_ik values
y_values = {(i, k): pyo.value(model.y_ik[i, k]) for (i, k) in model.valid_pairs}

print('Steady-state unconditional probabilities the system is in state i and decision k is made')
print('P{state = i and decision = k)')
display(y_values)

# Compute sum over k for each i
row_sums = {i: sum(y_values[i, k] for k in model.depths if (i, k) in y_values)
            for i in model.rows}

print(f'\nSteady_state probabilities π[i]):{row_sums}')

# Compute D_ik as normalized values
D_ik = {(i, k): int(y_values[i, k] / row_sums[i]) for (i, k) in y_values}

# Print the normalized decision values
print("\nNormalized Decision Variables D[i,k]:")
prev_i = None
for (i, k), val in D_ik.items():
    if i != prev_i:  # Check if 'i' changes
        if i != 0:
            print("\n")   # Print a new line
        prev_i = i    # Update previous i
    print(f"D[{i},{k}] = {val:.0f}")

#### Notes:
- The policy states to leave the machine as is (decision 1) in states 0 or 1,
- overhaul it (decision 2) in state 2,
- and replace it (decision 3) in state 3.

This is the same optimal policy identified through exhaustive enumeration in Section 19.2. (Used Grammarly to edit "Intro to Operations Research" by Hillier and Lieberman, Edition 10, Page 887, paragraph 2)

### LP Sensitivity Reports

Providing sensitivity analysis is a best practice with linear programming models; the Gurobi and GLPK exports below serve this purpose.

In [ ]:
import pyomo.environ as pyo

def solve_and_export_sensitivity(model, solver_name="gurobi", output_filename="sensit.sen"):
    """
    Solves the Pyomo model and exports sensitivity analysis if supported.
    
    Parameters:
    - model: Pyomo model instance
    - solver_name: 'gurobi' or 'glpk'
    - export_sensitivity: Boolean flag to enable sensitivity analysis export
    - output_filename: Filename for sensitivity analysis output
    """
    
    # Select Solver
    solver = pyo.SolverFactory(solver_name)

    output_filename = solver_name + "_" + output_filename
    
    # Ensure solver supports sensitivity analysis
    if solver_name not in ["gurobi", "glpk"]:
        print(f"Warning: Sensitivity analysis is not supported for {solver_name}. Skipping export.")

    # Solve model
    results = solver.solve(model, 
                           suffixes=["dual", "rc"])

    # Check if the solution is optimal
    if (results.solver.status != pyo.SolverStatus.ok) or (results.solver.termination_condition != pyo.TerminationCondition.optimal):
        print("Optimal solution not found. Sensitivity analysis unavailable.")
        return results

    if solver_name == "gurobi":
        # Add suffixes for reduced costs and shadow prices
        model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
        model.rc = pyo.Suffix(direction=pyo.Suffix.IMPORT)

        # Solve again to populate suffix values
        solver.solve(model,
                     suffixes=["dual", "rc"])

        # Save manually extracted sensitivity results
        with open(output_filename, "w") as f:
            f.write("Sensitivity Analysis (Dual Prices and Reduced Costs)\n")
            f.write("=" * 50 + "\n")

            # Shadow prices (dual values)
            f.write("Shadow Prices (Duals):\n")
            for c in model.component_objects(pyo.Constraint, active=True):
                for index in c:
                    f.write(f"{c.name}[{index}]: {model.dual[c[index]]}\n")

            f.write("\nReduced Costs:\n")
            for v in model.component_objects(pyo.Var, active=True):
                for index in v:
                    f.write(f"{v.name}[{index}]: {model.rc[v[index]]}\n")

        print(f"Sensitivity report saved to {output_filename}")

    elif solver_name == "glpk":
        # GLPK supports direct sensitivity analysis output
        model.write("model.lp")
        os.system(f"glpsol --lp model.lp --ranges {output_filename}")
        print(f"Sensitivity report saved to {output_filename}")

    return results


In [ ]:
solve_and_export_sensitivity(model, solver_name="glpk")

In [ ]:
solve_and_export_sensitivity(model, solver_name="gurobi")